# Predicting diamond prices

The aim of this project is to create a model that accurately predicts diamond prices

Packages required to run this piece of code:

1. Numpy
2. Pandas
3. Matplotlib
4. Sklearn
5. graphviz
6. Fastai0.7

## Key findings

1. Size is the most important: especially carat and y. However, probably high colinearity (have not checked yet)
2. Random forest can be used to predict this well. (although simpler linear models may also be alright)

## Importing libraries

Automatic reloading and inline plotting

In [ ]:
%load_ext autoreload
%autoreload 2


%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd

from fastai.imports import *
#from fastai.structured import *
#from structured import *
from fastai.tabular import *
from fastai import *
#from fastai_structured import *


from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics
from sklearn.model_selection import train_test_split, ParameterGrid, GridSearchCV

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

from scipy.cluster import hierarchy as hc

from fastai_structured import *

## Data preprocessing

#### Reading in data

In [ ]:
!ls ../input/

In [ ]:
PATH = "../input/"
!ls {PATH}

In [ ]:
## Training dataset
df_raw = pd.read_csv(f'{PATH}diamonds.csv', low_memory=False,
                    index_col = 'Unnamed: 0')

#### Checking data

In [ ]:
df_raw.head()

In [ ]:
df_raw.describe()

In [ ]:
plt.hist(list(df_raw.carat), bins=10, edgecolor='white')
plt.ylabel("Occurance")
plt.xlabel("Carats")
plt.title("Occurance of varying carats")

Obviously more people buy low carat diamonds than high carat diamond. This means the data may be inbalanced for prediction of high carat diamonds. While I may not have time to do this, I would recommend repeating the ****under-represented samples**** or using K fold cross validation cleverly to always include all of the rare samples

In [ ]:
new_index = ['Ideal', 'Premium', 'Very Good', 'Good', 'Fair']
plt.bar(df_raw.groupby('cut').count().iloc[:,0].reindex(new_index).index,
       df_raw.groupby('cut').count().iloc[:,0].reindex(new_index)/len(df_raw.cut))
plt.ylabel("Occurance")
plt.xlabel("Cuts")
plt.title("Occurance of varying cuts")

In [ ]:
df_raw.color.unique()

In [ ]:
new_index = ['E', 'F', 'G', 'H', 'I', 'J']
plt.bar(df_raw.groupby('color').count().iloc[:,0].reindex(new_index).index,
       df_raw.groupby('color').count().iloc[:,0].reindex(new_index)/len(df_raw.cut))
plt.ylabel("Occurance")
plt.xlabel("Color")
plt.title("Occurance of varying color")

In [ ]:
plt.hist(list(df_raw.depth), bins=100, edgecolor='white')
plt.ylabel("Occurance")
plt.xlabel("Depth")
plt.xlim(55,70)
plt.title("Occurance of varying depth")

In [ ]:
plt.hist(list(df_raw.table), bins=30, edgecolor='white')
plt.ylabel("Occurance")
plt.xlabel("Table")
plt.xlim(50,70)
plt.title("Occurance of varying table")

In [ ]:
plt.hist(list(df_raw.x), bins=30, edgecolor='white')
plt.ylabel("Occurance")
plt.xlabel("X")
plt.xlim(3,10)
plt.title("Occurance of varying x")

In [ ]:
plt.hist(list(df_raw.y), bins=100, edgecolor='white')
plt.ylabel("Occurance")
plt.xlabel("Y")
plt.xlim(3,10)
plt.title("Occurance of varying y")

In [ ]:
plt.hist(list(df_raw.z), bins=100, edgecolor='white')
plt.ylabel("Occurance")
plt.xlabel("Z")
plt.xlim(2,6)
plt.title("Occurance of varying z")

In [ ]:
plt.hist(list(df_raw.price), bins=100, edgecolor='white')
plt.ylabel("Occurance")
plt.xlabel("Price")
plt.title("Occurance of varying price")

Note that this is very skewed, not normal distribution

In [ ]:
df_raw['price'] = np.log(df_raw['price']); df_raw['price'].head()

In [ ]:
plt.hist(list(df_raw.price), bins=100, edgecolor='white')
plt.ylabel("Occurance")
plt.xlabel("Price")
plt.title("Occurance of varying price")

Still not normally distributed but much better!

Dealing with categorical data:

In [ ]:
train_cats(df_raw)
#apply_cats(df_test_raw,df_raw)

Check for NaNs

In [ ]:
(df_raw.isnull().sum().sort_values(ascending=False)/len(df_raw)).head(10)

No NaNs! Happy days!

In [ ]:
df, y, nas = proc_df(df_raw, 'price')

In [ ]:
X_train,X_valid,y_train,y_valid = train_test_split(df,y,test_size=0.25, random_state=42)

## Running a basic untuned RF 

In [ ]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

****R2 = 0.99****

## Looking at feature importance

In [ ]:
m = RandomForestRegressor(n_estimators=1, max_depth=3, bootstrap=False, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
import IPython
import graphviz

def mydraw_tree(t, df, size=10, ratio=0.6, precision=0):
    s=export_graphviz(t, out_file=None, feature_names=df.columns, filled=True,
                      special_characters=True, rotate=True, precision=precision)
    IPython.display.display(graphviz.Source(re.sub('Tree {',
       f'Tree {{ size={size}; ratio={ratio}', s)))

In [ ]:
## Draw tree is fastai
mydraw_tree(m.estimators_[0], X_train, precision=3)

Seems like carat and y are most important. Lets drop size dimensions to have a look:

In [ ]:
df_nosize = df.copy()

In [ ]:
df_nosize.columns

In [ ]:
df_nosize = df_nosize.drop(['carat','depth','table','x','y','z'],axis=1)

In [ ]:
df_nosize.head()

In [ ]:
X_train,X_valid,y_train,y_valid = train_test_split(df_nosize,y,test_size=0.25, random_state=42)

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

only R2=0.1

In [ ]:
X_train

In [ ]:
m = RandomForestRegressor(n_estimators=1, max_depth=3, bootstrap=False, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
import IPython
import graphviz

def mydraw_tree(t, df, size=10, ratio=0.6, precision=0):
    s=export_graphviz(t, out_file=None, feature_names=df.columns, filled=True,
                      special_characters=True, rotate=True, precision=precision)
    IPython.display.display(graphviz.Source(re.sub('Tree {',
       f'Tree {{ size={size}; ratio={ratio}', s)))

In [ ]:
## Draw tree is fastai
mydraw_tree(m.estimators_[0], X_train, precision=3)

Clarity and color seems to be the most important non size feature

## Tuning RF

In [ ]:
X_train,X_valid,y_train,y_valid = train_test_split(df,y,test_size=0.25, random_state=42)

In [ ]:
m = RandomForestRegressor(n_estimators=100,n_jobs=-1,oob_score=True)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
## Each tree is found in m.estimators_

preds = np.stack([t.predict(X_valid) for t in m.estimators_])
preds[:,0], np.mean(preds[:,0]), y_valid[0]

In [ ]:
plt.plot([metrics.r2_score(y_valid, np.mean(preds[:i+1], axis=0)) for i in range(100)]);

only marginal improvement after 20 estimators

In [ ]:
param_grid = {
    'min_samples_leaf': [1, 5, 10],
    'max_features': [0.5, 1],
    'n_estimators': [20],
    'n_jobs': [-1],
    'random_state': [42]
}

m = RandomForestRegressor(n_estimators=20)

grid_search = GridSearchCV(m, param_grid=param_grid, cv=5, iid=False,
                           verbose=1, scoring='neg_mean_squared_error');
grid_search.fit(X_train, y_train);
#print(grid_search.cv_results_)

In [ ]:
print(grid_search.best_score_)

In [ ]:
print(grid_search.best_params_)

In [ ]:
myscoredf = pd.DataFrame(grid_search.cv_results_)[['param_min_samples_leaf','param_max_features','mean_test_score']]; myscoredf.head(10)

In [ ]:
myscoredf = myscoredf.pivot('param_min_samples_leaf','param_max_features','mean_test_score')

In [ ]:
ax = sns.heatmap(myscoredf, annot=True, fmt=".5g", cmap=cm.coolwarm)

In [ ]:
m = RandomForestRegressor(n_estimators=20,
                          min_samples_leaf=grid_search.best_params_['min_samples_leaf'],
                          max_features=grid_search.best_params_['max_features'],
                          n_jobs=-1)
m.fit(X_train, y_train);
print_score(m)

## Looking at feature importance

In [ ]:
fi = rf_feat_importance(m, df); fi[:10]

In [ ]:
fi.plot('cols', 'imp', figsize=(10,6), legend=False);

In [ ]:
def plot_fi(fi): return fi.plot('cols', 'imp', 'barh', figsize=(12,7), legend=False)
plot_fi(fi[:30]);

Note that y, x, z and carat are correlated which a dendrogram would probably show!

## Future work

1. Check how model fares with high carat diamonds (may have less samples)
2. Given the R2 and how everything is based on size, a linear model with transformed (logged) price may also work well
3. The interaction between x, y, z and carat may be interesting to explore